In [1]:
import scipy.io as spio
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from tsfresh import extract_features, extract_relevant_features, select_features
import numpy as np
import re
from bottleneck import move_mean

def find_regex_index(regex, dictionary):
    return [index for index, value in dictionary.items() if re.match(regex, index)][0]


In [3]:
mat_data_raw = spio.loadmat('D:\\facul\\eeg_raw_data\\1\\15_20150508.mat')

mat_data_raw.keys()

dict_keys(['__header__', '__version__', '__globals__', 'zjy_eeg1', 'zjy_eeg2', 'zjy_eeg3', 'zjy_eeg4', 'zjy_eeg5', 'zjy_eeg6', 'zjy_eeg7', 'zjy_eeg8', 'zjy_eeg9', 'zjy_eeg10', 'zjy_eeg11', 'zjy_eeg12', 'zjy_eeg13', 'zjy_eeg14', 'zjy_eeg15', 'zjy_eeg16', 'zjy_eeg17', 'zjy_eeg18', 'zjy_eeg19', 'zjy_eeg20', 'zjy_eeg21', 'zjy_eeg22', 'zjy_eeg23', 'zjy_eeg24'])

In [5]:
mat_data_raw['zjy_eeg1'].shape

(62, 33601)

In [6]:
mat_data_raw['zjy_eeg1'][0][:100]

array([ 36.62705421, -24.58691597,  53.07793617, -28.40161324,
        39.10064697, -24.73592758,  50.15730858, -36.59725189,
        29.6831131 , -34.481287  ,  45.29953003, -35.16674042,
        33.70642662, -34.60049629,  41.87226295, -43.54119301,
        23.09679985, -46.96846008,  32.66334534, -50.81295967,
        27.6863575 , -34.92832184,  30.84540367, -48.75659943,
        26.16643906, -39.21985626,  39.48807716, -45.388937  ,
        18.53704453, -37.90855408,  36.92507744, -44.91209984,
        30.51757812, -41.90206528,  36.53764725, -43.21336746,
        24.79553223, -43.00475121,  35.37535667, -43.03455353,
        33.1401825 , -32.2163105 ,  40.02451897, -45.86577415,
        30.2195549 , -40.50135612,  31.38184547, -45.50814629,
        23.1564045 , -36.65685654,  41.27621651, -38.83242607,
        27.83536911, -40.62056541,  33.61701965, -43.83921623,
        27.4181366 , -36.23962402,  38.86222839, -30.27915955,
        47.32608795, -19.28210258,  46.37241364, -34.09

In [8]:

# electrode order on mat file
electrode_order_names = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']

electrode_names = ['T7', 'T8', 'FT7', 'FT8', 'TP7', 'TP8']
electrode_names_indices = [i for i, x in enumerate(electrode_order_names) if x in electrode_names]

mat_subset = [val for i, val in enumerate(mat_data_raw['zjy_eeg1']) if i in electrode_names_indices]


df = pd.DataFrame(np.array(mat_subset).T)
sf = 200
df['empty_channel_1'] = "0.00"
df['empty_channel_2'] = "0.00"
df['accelerometer_X'] = "0.000"
df['accelerometer_Y'] = "0.000"
df['accelerometer_Z'] = "0.000"
df['timestamp'] = ""
df


,0,1,2,3,4,5,empty_channel_1,empty_channel_2,accelerometer_X,accelerometer_Y,accelerometer_Z,timestamp
0,30.010939,40.352345,31.232834,38.832426,44.554472,24.914742,0.00,0.00,0.000,0.000,0.000,
1,-35.256147,-17.553568,-33.915043,-16.689301,-12.665987,-20.593405,0.00,0.00,0.000,0.000,0.000,
2,50.097704,63.389540,51.796436,33.736229,53.137541,30.606985,0.00,0.00,0.000,0.000,0.000,
3,-37.670135,-25.004148,-35.613775,-23.216009,-20.980835,-32.663345,0.00,0.00,0.000,0.000,0.000,
4,33.587217,41.872263,35.375357,41.097403,34.093857,20.503998,0.00,0.00,0.000,0.000,0.000,
...,...,...,...,...,...,...,...,...,...,...,...,...
33596,-2.533197,15.497208,-8.583069,-2.861023,-0.596046,14.722347,0.00,0.00,0.000,0.000,0.000,
33597,51.647425,74.326992,45.776367,27.209520,64.224005,80.049038,0.00,0.00,0.000,0.000,0.000,
33598,-5.364418,-0.476837,-10.967255,14.632940,14.930964,22.917986,0.00,0.00,0.000,0.000,0.000,
33599,58.770180,66.578388,50.902367,72.479248,75.340271,75.399876,0.00,0.00,0.000,0.000,0.000,


In [11]:
# filename = './test_SAD_LABEL_INDEX_1.txt'
filename = './user_15_trial_0_label_negative.txt'


In [12]:
df.to_csv(filename, header=False, sep=',')


f = open(filename,'r')
filedata = f.read()
f.close()

newdata = filedata.replace(","," ,")


fileheader = ('\n').join([
    "%OpenBCI Raw EEG Data",
    "%Number of channels = " + str(len(electrode_names)),
    "%Sample Rate = " + str(sf) + ' Hz',
    "%First Column = SampleIndex",
    "%Last Column = Timestamp ",
    "%Other Columns = EEG data in microvolts followed by Accel Data (in G) interleaved with Aux Data",
    ""
])

f = open(filename,'w')
f.write(fileheader)
f.write(newdata)
f.close()